In [2]:
import numpy as npa
import pandas as pd

import pytz
import locale
from datetime import datetime

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.appName('ThExam').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/02 15:19:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/02 15:19:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/02 15:19:17 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
local_timezone = pytz.timezone('Asia/Bangkok')
execute_time = datetime.now(local_timezone)
year = execute_time.year
month = execute_time.month
day = execute_time.day

print(execute_time)
print("year:{} | month={} | day={}".format(year,month,day))

2024-03-02 15:19:17.983632+07:00
year:2024 | month=3 | day=2


## Load to Postgres 

In [10]:
schema = StructType([
    StructField("user",StringType(),True),
    StructField("timestamp",TimestampType(),True),
    StructField("hours",FloatType(),True),
    StructField("project",StringType(),True),
  ])

In [11]:
TABLE = 'dailycheckins'

In [12]:
path = "../data/state=test/table={}/year={}/month={}/day={}/".format(TABLE,year,month,day)
df = spark.read.schema(schema).csv(path, header=True)

In [13]:
df.describe().toPandas()

,summary,user,hours,project
0,count,0,5,5
1,mean,None,2.95,None
2,stddev,None,1.2298373876248843,None
3,min,None,1.0,bizdev
4,max,None,4.0,transit


In [17]:
df = df.withColumnRenamed('user', 'username').drop('user')

In [18]:
df.columns

['username', 'timestamp', 'hours', 'project']

In [19]:
aurora_postgres_url = "jdbc:postgresql://database-1.cluster-cqnnuqtyqggk.ap-southeast-1.rds.amazonaws.com:5432/tm"
aurora_postgres_properties = {
    "user": "prachya",
    "password": "prachya2024",
    "driver": "org.postgresql.Driver"
}

In [20]:
df.limit(10).write\
.format("jdbc")\
.option("url", aurora_postgres_url)\
.option("dbtable", "dailycheckin")\
.option("user", aurora_postgres_properties["user"])\
.option("password", aurora_postgres_properties["password"])\
.option("driver", aurora_postgres_properties["driver"])\
.mode("append")\
.save()

Py4JJavaError: An error occurred while calling o82.save.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:476)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:589)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:522)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:246)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:250)
	at org

In [25]:
'table%3Ddailycheckin/year%3D2024/month%3D3/day%3D2/dailycheckins.csv'.split('.')[0].split('/')[-1]

'dailycheckins'